**Denial constraints** are really powerful in finding complex metadata in your data. Let's try them with [Desbordante](https://github.com/Desbordante/desbordante-core)!

# Install necessary dependencies

Firstly, let's download and import necessary libraries:

In [ ]:
!pip install desbordante==2.3.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 32.3 MB/s eta 0:00:00


Desbordante library will be used for discovery of denial constraints and Pandas library will be used for visualising the data:

In [ ]:
import desbordante
import pandas as pd

Let's download example data:

In [ ]:
!wget https://raw.githubusercontent.com/Desbordante/desbordante-core/refs/heads/main/examples/datasets/taxes.csv

--2025-03-20 16:58:54--  https://raw.githubusercontent.com/Desbordante/desbordante-core/refs/heads/main/examples/datasets/taxes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 181 [text/plain]
Saving to: ‘taxes.csv’

taxes.csv           100%[===================>]     181  --.-KB/s    in 0s      

2025-03-20 16:58:54 (8.69 MB/s) - ‘taxes.csv’ saved [181/181]



# Understanding denial constraints

A denial constraint (DC) is a statement that says: "For all pairs of different rows in a table, it should never happen that some condition holds."

Formally, DC $\varphi$ is a conjunction of predicates of the following form:
$\forall s, t \in R, s \neq t: \neg(p_1 \wedge \cdots \wedge p_m)$.

For example, let's create a small table:

In [ ]:
data = {'Name': ['Alice', 'Bob', 'Carol'],
        'Grade': [3, 4, 4],
        'Salary': [3000, 4000, 4000]}

small_dataset = pd.DataFrame(data)
small_dataset

,Name,Grade,Salary
0,Alice,3,3000
1,Bob,4,4000
2,Carol,4,4000


A possible DC here is: ¬{ t.Grade == s.Grade ∧ t.Salary != s.Salary }. This means: "It should never happen that two people have the same grade but different salaries.", or in other words, if two rows share the same Grade, they must share the same Salary.

As we can see from the table, only tuple pair (1, 2) satisfies the first predicate (its values on column Grade are equal):

In [ ]:
def color_cells(x):
  df1=pd.DataFrame('',index=x.index,columns=x.columns)
  df1.iloc[1,1]='color:green;font-weight:bold'
  df1.iloc[2,1]='color:green;font-weight:bold'
  df1.iloc[1,2]='color:red;font-weight:bold'
  df1.iloc[2,2]='color:red;font-weight:bold'
  return df1

small_dataset.style.apply(color_cells,axis=None)

,Name,Grade,Salary
0,Alice,3,3000
1,Bob,4,4000
2,Carol,4,4000


This tuple pair doesn't satisfy the second predicate (its values on column Salary are also equal), thus the DC holds in the table.

Sometimes, we allow a DC to hold approximately, which means a small number of row pairs might violate it. The measure used for that is the $g_1$ metric. Roughly, the $g_1$ metric checks what fraction of all row pairs violates the DC, and if that fraction is lower than a chosen threshold, we consider the DC 'valid enough.'

# Mining denial constraints

Let's have a look at the dataset:

In [ ]:
dataset = pd.read_csv('taxes.csv')
dataset

,State,Salary,FedTaxRate
0,NewYork,3000,0.20
1,NewYork,4000,0.25
2,NewYork,5000,0.30
3,Wisconsin,5000,0.15
4,Wisconsin,6000,0.20
5,Wisconsin,4000,0.10
6,Texas,1000,0.15
7,Texas,2000,0.25
8,Texas,3000,0.30


The dataset contains some information about salaries and taxes in different US states.

Now, let's find DCs using Desbordante:

In [ ]:
algo = desbordante.dc.algorithms.Default()
algo.load_data(table=dataset)
algo.execute(evidence_threshold=0, shard_length=0)
exact_dcs = algo.get_dcs()

print("Discovered DCs:")
for dc in exact_dcs:
    print(dc)

Discovered DCs:
¬{ t.State == s.State ∧ t.Salary <= s.Salary ∧ t.FedTaxRate >= s.FedTaxRate }
¬{ t.Salary == s.Salary ∧ t.FedTaxRate == s.FedTaxRate }
¬{ t.State == s.State ∧ t.FedTaxRate == s.FedTaxRate }
¬{ t.State == s.State ∧ t.Salary == s.Salary }


The DC mining algorithm has found four DCs!

We have two parameters in Desbordante's DC mining algorithm:

1.   evidence_threshold: This sets the fraction of row pairs that must satisfy the DC for it to be considered valid. A value of 0 means exact DC mining (no violations allowed).
2.   shard_length: This splits the dataset into row "shards" for parallelization. A value of 0 means no split, so the entire dataset is processed at once.



Note the following DC we found:

¬{ t.State == s.State ∧ t.Salary <= s.Salary ∧ t.FedTaxRate >= s.FedTaxRate }.

It states that for all people in the same state, the person with a higher salary
should have a higher tax rate. No pairs of rows should violate that rule.

# Verifying DCs

The algorithm for DC verification can check whether a given exact DC holds in the table.

Let's check the above DC using Desbordante:

In [ ]:
DC = "!(t.State == s.State and t.Salary <= s.Salary and t.FedTaxRate >= s.FedTaxRate)"

verifier = desbordante.dc_verification.algorithms.Default()
verifier.load_data(table=dataset)
verifier.execute(denial_constraint=DC)

if(verifier.dc_holds()):
  print(f"DC {DC} holds.")
else:
  print(f"DC {DC} doesn't hold.")

DC !(t.State == s.State and t.Salary <= s.Salary and t.FedTaxRate >= s.FedTaxRate) holds.


As we can see, the above DC indeed holds in the table.

# Mining approximate DCs

Now let's mine approximate DCs (ADCs) by setting evidence_threshold to 0.5.
This means we only require that at least half of all row pairs satisfy each DC (according to $g_1$).

In [ ]:
algo.execute(evidence_threshold=0.5, shard_length=0)
approximate_dcs = algo.get_dcs()

print("Discovered ADCs:")
for dc in approximate_dcs:
    print(dc)

Discovered ADCs:
¬{ t.Salary <= s.Salary ∧ t.FedTaxRate <= s.FedTaxRate }
¬{ t.Salary <= s.Salary ∧ t.FedTaxRate >= s.FedTaxRate }
¬{ t.Salary <= s.Salary ∧ t.FedTaxRate != s.FedTaxRate }
¬{ t.State != s.State ∧ t.FedTaxRate <= s.FedTaxRate }
¬{ t.State != s.State ∧ t.Salary <= s.Salary }
¬{ t.FedTaxRate < s.FedTaxRate }
¬{ t.Salary < s.Salary }
¬{ t.FedTaxRate == s.FedTaxRate }
¬{ t.Salary == s.Salary }
¬{ t.State == s.State }


Here, for example, the $g_1$ metric values for a few approximate DCs are:

¬{ t.Salary <= s.Salary ∧ t.FedTaxRate <= s.FedTaxRate } → 0.486111

¬{ t.Salary <= s.Salary ∧ t.FedTaxRate >= s.FedTaxRate } → 0.458333

¬{ t.State == s.State } → 0.25

Note: A smaller $g_1$ value means fewer violations, making the DC more close to exact.

In real scenarios, exact DCs may be too rigid. Allowing a small fraction of violations is often a practical compromise, but setting a very high threshold quickly becomes meaningless since it would permit too many inconsistencies. The best threshold often depends on how 'dirty' the data is; datasets with more inconsistencies may require a higher threshold to capture meaningful DCs.

Now, let's add a new row to the dataset:

In [ ]:
dataset.loc[9] = ["Texas", 5000, 0.05]
dataset

,State,Salary,FedTaxRate
0,NewYork,3000,0.20
1,NewYork,4000,0.25
2,NewYork,5000,0.30
3,Wisconsin,5000,0.15
4,Wisconsin,6000,0.20
5,Wisconsin,4000,0.10
6,Texas,1000,0.15
7,Texas,2000,0.25
8,Texas,3000,0.30
9,Texas,5000,0.05


Let's run the algorithm again and see how the exact DC mining changes due to this additional record:

In [ ]:
algo1 = desbordante.dc.algorithms.Default()
algo1.load_data(table=dataset)
algo1.execute(evidence_threshold=0, shard_length=0)
new_exact_dcs = algo1.get_dcs()

print("Discovered DCs:")
for dc in new_exact_dcs:
    print(dc)

Discovered DCs:
¬{ t.Salary == s.Salary ∧ t.FedTaxRate == s.FedTaxRate }
¬{ t.State == s.State ∧ t.FedTaxRate == s.FedTaxRate }
¬{ t.State == s.State ∧ t.Salary == s.Salary }


Let's recall the result for the old dataset:

In [ ]:
for dc in exact_dcs:
    print(dc)

¬{ t.State == s.State ∧ t.Salary <= s.Salary ∧ t.FedTaxRate >= s.FedTaxRate }
¬{ t.Salary == s.Salary ∧ t.FedTaxRate == s.FedTaxRate }
¬{ t.State == s.State ∧ t.FedTaxRate == s.FedTaxRate }
¬{ t.State == s.State ∧ t.Salary == s.Salary }


We can see that the DC ¬{ t.State == s.State ∧ t.Salary <= s.Salary ∧ t.FedTaxRate >= s.FedTaxRate } no longer appears because of the violation introduced by the added record.

This record introduces a scenario that breaks this DC, stating "the person with a higher salary should have a higher tax rate," because there are now people in Texas with a higher salary but a lower tax rate.

Those violations occur in pairs like (6, 9), (7, 9), (8, 9),
where each number is a record index in the dataset:

In [ ]:
def color_cells(x):
  df1=pd.DataFrame('',index=x.index,columns=x.columns)
  df1.iloc[6,1]='color:green;font-weight:bold'
  df1.iloc[9,1]='color:green;font-weight:bold'
  df1.iloc[6,2]='color:red;font-weight:bold'
  df1.iloc[9,2]='color:red;font-weight:bold'
  return df1

dataset.style.apply(color_cells,axis=None)

,State,Salary,FedTaxRate
0,NewYork,3000,0.200000
1,NewYork,4000,0.250000
2,NewYork,5000,0.300000
3,Wisconsin,5000,0.150000
4,Wisconsin,6000,0.200000
5,Wisconsin,4000,0.100000
6,Texas,1000,0.150000
7,Texas,2000,0.250000
8,Texas,3000,0.300000
9,Texas,5000,0.050000


Let's check this DC again:

In [ ]:
DC = "!(t.State == s.State and t.Salary <= s.Salary and t.FedTaxRate >= s.FedTaxRate)"

verifier = desbordante.dc_verification.algorithms.Default()
verifier.load_data(table=dataset)
verifier.execute(denial_constraint=DC)

if(verifier.dc_holds()):
  print(f"DC {DC} holds.")
else:
  print(f"DC {DC} doesn't hold.")

DC !(t.State == s.State and t.Salary <= s.Salary and t.FedTaxRate >= s.FedTaxRate) doesn't hold.


The DC doesn't hold, as we expected.

# Conclusion

If you are reading this, then you have learnt about denial constraints. Congratulations!

We have explored data and found that for all people living in the same state, the person with a higher salary should also have a higher tax rate. We have also learnt about the approximation of DCs with $g_1$ metric and why allowing violations may be useful.


If you wish to find these patterns in your data, now you know how to do it 🙂
Also, you can learn more about other pattern types presented in [Desbordante](https://github.com/Desbordante/desbordante-core).